## Train a character-level GPT on some text data

The inputs here are simple text files, which we chop up to individual characters and then train GPT on. So you could say this is a char-transformer instead of a char-rnn. Doesn't quite roll off the tongue as well. In this example we will feed it some Shakespeare, which we'll get it to predict character-level.

In [1]:
from allennlp.data.token_indexers import TokenIndexer, PretrainedTransformerIndexer
from allennlp.data.tokenizers import Token, Tokenizer, PretrainedTransformerTokenizer

import nltk
#nltk.download('punkt')
import numpy as np
from os import listdir
from os.path import join as pathjoin
import torch
import torch.nn as nn
from torch.nn import functional as F
import tqdm

from minGPT.mingpt.model import GPT, GPTConfig
from minGPT.mingpt.trainer import Trainer, TrainerConfig
# make deterministic
from minGPT.mingpt.utils import sample, set_seed
set_seed(42)

In [2]:
DATA_DIR = '/home/mlepekhin/data'
MODELS_DIR = '/home/mlepekhin/models'
transformer_model = 'bert-base-cased'

In [3]:
import math
from torch.utils.data import Dataset


def detokenize(tokens):
    return ' '.join([str(x) for x in tokens[1:-1]]).replace(' ##', '')

class BPEDataset(Dataset):
    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx + self.block_size + 1]
        # encode every character to an integer
        dix = [self.stoi[s] for s in chunk]
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y


In [4]:
block_size = 128
tokenizer = PretrainedTransformerTokenizer(transformer_model)
#indexer = PretrainedTransformerIndexer(transformer_model)

In [5]:
def train_gpt_generator(train_text_file, state_dict_file, n_layer=8, n_head=8, n_embd=512,
                        max_epochs=2, batch_size=256):
    text_sentences = nltk.tokenize.sent_tokenize(open(train_text_file, 'r').read())
    tokens = np.concatenate([tokenizer.tokenize(sent)[1:-1] for sent in text_sentences])
    tokens = [str(token) for token in tokens]
    train_dataset = BPEDataset(tokens, block_size) 
    
    mconf = GPTConfig(
        train_dataset.vocab_size, train_dataset.block_size,
        n_layer=n_layer, n_head=n_head, n_embd=n_embd
    )
    model = GPT(mconf)
    tconf = TrainerConfig(
        max_epochs=max_epochs, batch_size=batch_size, learning_rate=6e-4,
        lr_decay=True, warmup_tokens=batch_size*20, final_tokens=2*len(train_dataset)*block_size,
        num_workers=2
    )
    trainer = Trainer(model, train_dataset, None, tconf)
    trainer.train()
    torch.save(model.state_dict(), state_dict_file)

In [6]:
GENRE_DATA_DIR = '/home/mlepekhin/data/genre'
GPT_MODELS_DIR = '/home/mlepekhin/models/mini_gpt_bpe/'
LANG = 'en'

In [7]:
#train_gpt_generator(
#        pathjoin(GENRE_DATA_DIR, LANG, 'A1.txt'),
#        pathjoin(GPT_MODELS_DIR, LANG, 'A1')
#)

In [ ]:
for train_text_file in tqdm.tqdm(listdir(pathjoin(GENRE_DATA_DIR, LANG))):
    label = train_text_file[:-4]
    train_gpt_generator(
        pathjoin(GENRE_DATA_DIR, LANG, train_text_file),
        pathjoin(GPT_MODELS_DIR, LANG, label)
    )

  0%|          | 0/11 [00:00<?, ?it/s]

data has 191478 characters, 11294 unique.



  0%|          | 0/748 [00:00<?, ?it/s]/home/mlepekhin/anaconda3/envs/mlepekhin_research/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '

epoch 1 iter 0: train loss 9.44247. lr 5.999995e-04:   0%|          | 0/748 [00:07<?, ?it/s]
epoch 1 iter 0: train loss 9.44247. lr 5.999995e-04:   0%|          | 1/748 [00:07<1:32:58,  7.47s/it]
epoch 1 iter 1: train loss 8.68254. lr 5.999977e-04:   0%|          | 1/748 [00:07<1:32:58,  7.47s/it]
epoch 1 iter 1: train loss 8.68254. lr 5.999977e-04:   0%|          | 2/748 [00:07<1:06:13,  5.33s/it]
epoch 1 iter 2: train loss 8.22270. lr 5.999946e-04:   0%|          | 2/748 [00:08<1:06:13,  5.33s/it]
epoch 1 iter 2: train loss 8.22270. lr 5.999946e-04:   0%|          | 3/748 [00:08<47:30,  3.83s/it]  
epoch 1 iter 3: train loss 7.95068. lr 

epoch 1 iter 75: train loss 5.67482. lr 5.961967e-04:  10%|█         | 76/748 [00:31<03:33,  3.14it/s]
epoch 1 iter 76: train loss 5.71231. lr 5.960959e-04:  10%|█         | 76/748 [00:31<03:33,  3.14it/s]
epoch 1 iter 76: train loss 5.71231. lr 5.960959e-04:  10%|█         | 77/748 [00:31<03:33,  3.15it/s]
epoch 1 iter 77: train loss 5.68845. lr 5.959939e-04:  10%|█         | 77/748 [00:32<03:33,  3.15it/s]
epoch 1 iter 77: train loss 5.68845. lr 5.959939e-04:  10%|█         | 78/748 [00:32<03:54,  2.85it/s]
epoch 1 iter 78: train loss 5.63510. lr 5.958905e-04:  10%|█         | 78/748 [00:32<03:54,  2.85it/s]
epoch 1 iter 78: train loss 5.63510. lr 5.958905e-04:  11%|█         | 79/748 [00:32<03:47,  2.94it/s]
epoch 1 iter 79: train loss 5.70678. lr 5.957859e-04:  11%|█         | 79/748 [00:32<03:47,  2.94it/s]
epoch 1 iter 79: train loss 5.70678. lr 5.957859e-04:  11%|█         | 80/748 [00:32<03:42,  3.00it/s]
epoch 1 iter 80: train loss 5.65166. lr 5.956799e-04:  11%|█         | 80

epoch 1 iter 151: train loss 4.95283. lr 5.848522e-04:  20%|██        | 151/748 [00:55<03:10,  3.13it/s]
epoch 1 iter 151: train loss 4.95283. lr 5.848522e-04:  20%|██        | 152/748 [00:55<03:10,  3.14it/s]
epoch 1 iter 152: train loss 4.94371. lr 5.846537e-04:  20%|██        | 152/748 [00:56<03:10,  3.14it/s]
epoch 1 iter 152: train loss 4.94371. lr 5.846537e-04:  20%|██        | 153/748 [00:56<03:09,  3.13it/s]
epoch 1 iter 153: train loss 4.94180. lr 5.844540e-04:  20%|██        | 153/748 [00:56<03:09,  3.13it/s]
epoch 1 iter 153: train loss 4.94180. lr 5.844540e-04:  21%|██        | 154/748 [00:56<03:09,  3.13it/s]
epoch 1 iter 154: train loss 4.92770. lr 5.842531e-04:  21%|██        | 154/748 [00:56<03:09,  3.13it/s]
epoch 1 iter 154: train loss 4.92770. lr 5.842531e-04:  21%|██        | 155/748 [00:56<03:09,  3.13it/s]
epoch 1 iter 155: train loss 4.93664. lr 5.840509e-04:  21%|██        | 155/748 [00:57<03:09,  3.13it/s]
epoch 1 iter 155: train loss 4.93664. lr 5.840509e-04: 

...

epoch 1 iter 77: train loss 5.85716. lr 5.969192e-04:   9%|▉         | 77/853 [00:32<05:14,  2.47it/s]
epoch 1 iter 77: train loss 5.85716. lr 5.969192e-04:   9%|▉         | 78/853 [00:32<05:15,  2.46it/s]
epoch 1 iter 78: train loss 5.82457. lr 5.968397e-04:   9%|▉         | 78/853 [00:32<05:15,  2.46it/s]
epoch 1 iter 78: train loss 5.82457. lr 5.968397e-04:   9%|▉         | 79/853 [00:32<05:16,  2.45it/s]
epoch 1 iter 79: train loss 5.84702. lr 5.967592e-04:   9%|▉         | 79/853 [00:32<05:16,  2.45it/s]
epoch 1 iter 79: train loss 5.84702. lr 5.967592e-04:   9%|▉         | 80/853 [00:32<05:19,  2.42it/s]
epoch 1 iter 80: train loss 5.84464. lr 5.966777e-04:   9%|▉         | 80/853 [00:33<05:19,  2.42it/s]
epoch 1 iter 80: train loss 5.84464. lr 5.966777e-04:   9%|▉         | 81/853 [00:33<05:17,  2.43it/s]
epoch 1 iter 81: train loss 5.78319. lr 5.965951e-04:   9%|▉         | 81/853 [00:33<05:17,  2.43it/s]
epoch 1 iter 81: train loss 5.78319. lr 5.965951e-04:  10%|▉         | 82

epoch 1 iter 152: train loss 4.74106. lr 5.881811e-04:  18%|█▊        | 153/853 [01:03<04:45,  2.45it/s]
epoch 1 iter 153: train loss 4.67961. lr 5.880270e-04:  18%|█▊        | 153/853 [01:03<04:45,  2.45it/s]
epoch 1 iter 153: train loss 4.67961. lr 5.880270e-04:  18%|█▊        | 154/853 [01:03<04:44,  2.45it/s]
epoch 1 iter 154: train loss 4.74189. lr 5.878719e-04:  18%|█▊        | 154/853 [01:03<04:44,  2.45it/s]
epoch 1 iter 154: train loss 4.74189. lr 5.878719e-04:  18%|█▊        | 155/853 [01:03<04:43,  2.46it/s]
epoch 1 iter 155: train loss 4.76004. lr 5.877158e-04:  18%|█▊        | 155/853 [01:04<04:43,  2.46it/s]
epoch 1 iter 155: train loss 4.76004. lr 5.877158e-04:  18%|█▊        | 156/853 [01:04<04:42,  2.47it/s]
epoch 1 iter 156: train loss 4.62053. lr 5.875588e-04:  18%|█▊        | 156/853 [01:04<04:42,  2.47it/s]
epoch 1 iter 156: train loss 4.62053. lr 5.875588e-04:  18%|█▊        | 157/853 [01:04<05:07,  2.26it/s]
epoch 1 iter 157: train loss 4.60497. lr 5.874007e-04: 

epoch 1 iter 227: train loss 3.41137. lr 5.739481e-04:  27%|██▋       | 228/853 [01:34<04:16,  2.44it/s]
epoch 1 iter 228: train loss 3.54570. lr 5.737223e-04:  27%|██▋       | 228/853 [01:34<04:16,  2.44it/s]
epoch 1 iter 228: train loss 3.54570. lr 5.737223e-04:  27%|██▋       | 229/853 [01:34<04:16,  2.43it/s]
epoch 1 iter 229: train loss 3.33743. lr 5.734956e-04:  27%|██▋       | 229/853 [01:35<04:16,  2.43it/s]
epoch 1 iter 229: train loss 3.33743. lr 5.734956e-04:  27%|██▋       | 230/853 [01:35<04:15,  2.44it/s]
epoch 1 iter 230: train loss 3.32224. lr 5.732679e-04:  27%|██▋       | 230/853 [01:35<04:15,  2.44it/s]
epoch 1 iter 230: train loss 3.32224. lr 5.732679e-04:  27%|██▋       | 231/853 [01:35<04:15,  2.44it/s]
epoch 1 iter 231: train loss 3.39159. lr 5.730394e-04:  27%|██▋       | 231/853 [01:36<04:15,  2.44it/s]
epoch 1 iter 231: train loss 3.39159. lr 5.730394e-04:  27%|██▋       | 232/853 [01:36<04:18,  2.40it/s]
epoch 1 iter 232: train loss 3.36235. lr 5.728099e-04: 

epoch 1 iter 302: train loss 2.24055. lr 5.544915e-04:  36%|███▌      | 303/853 [02:06<03:47,  2.42it/s]
epoch 1 iter 303: train loss 2.25090. lr 5.541984e-04:  36%|███▌      | 303/853 [02:07<03:47,  2.42it/s]
epoch 1 iter 303: train loss 2.25090. lr 5.541984e-04:  36%|███▌      | 304/853 [02:07<03:45,  2.43it/s]
epoch 1 iter 304: train loss 2.23993. lr 5.539044e-04:  36%|███▌      | 304/853 [02:07<03:45,  2.43it/s]
epoch 1 iter 304: train loss 2.23993. lr 5.539044e-04:  36%|███▌      | 305/853 [02:07<03:44,  2.44it/s]
epoch 1 iter 305: train loss 2.20106. lr 5.536095e-04:  36%|███▌      | 305/853 [02:07<03:44,  2.44it/s]
epoch 1 iter 305: train loss 2.20106. lr 5.536095e-04:  36%|███▌      | 306/853 [02:07<03:43,  2.45it/s]
epoch 1 iter 306: train loss 2.21969. lr 5.533138e-04:  36%|███▌      | 306/853 [02:08<03:43,  2.45it/s]
epoch 1 iter 306: train loss 2.21969. lr 5.533138e-04:  36%|███▌      | 307/853 [02:08<03:42,  2.45it/s]
epoch 1 iter 307: train loss 2.16731. lr 5.530172e-04: 

epoch 1 iter 377: train loss 1.38811. lr 5.301824e-04:  44%|████▍     | 378/853 [02:38<03:29,  2.27it/s]
epoch 1 iter 378: train loss 1.39569. lr 5.298275e-04:  44%|████▍     | 378/853 [02:38<03:29,  2.27it/s]
epoch 1 iter 378: train loss 1.39569. lr 5.298275e-04:  44%|████▍     | 379/853 [02:38<03:26,  2.30it/s]
epoch 1 iter 379: train loss 1.27486. lr 5.294718e-04:  44%|████▍     | 379/853 [02:38<03:26,  2.30it/s]
epoch 1 iter 379: train loss 1.27486. lr 5.294718e-04:  45%|████▍     | 380/853 [02:38<03:23,  2.33it/s]
epoch 1 iter 380: train loss 1.29384. lr 5.291154e-04:  45%|████▍     | 380/853 [02:39<03:23,  2.33it/s]
epoch 1 iter 380: train loss 1.29384. lr 5.291154e-04:  45%|████▍     | 381/853 [02:39<03:19,  2.36it/s]
epoch 1 iter 381: train loss 1.35344. lr 5.287581e-04:  45%|████▍     | 381/853 [02:39<03:19,  2.36it/s]
epoch 1 iter 381: train loss 1.35344. lr 5.287581e-04:  45%|████▍     | 382/853 [02:39<03:17,  2.39it/s]
epoch 1 iter 382: train loss 1.31922. lr 5.284001e-04: 

In [15]:
from minGPT.mingpt.utils import sample

def generate_dataset(train_text_file, state_dict_file, n_layer=8, n_head=8, n_embd=512,
                     texts_count=1, text_len=100):
    text_sentences = nltk.tokenize.sent_tokenize(open(train_text_file, 'r').read())
    tokens = np.concatenate([tokenizer.tokenize(sent)[1:-1] for sent in text_sentences])
    tokens = [str(token) for token in tokens]
    train_dataset = BPEDataset(tokens, block_size) 
    
    mconf = GPTConfig(
        train_dataset.vocab_size, train_dataset.block_size,
        n_layer=n_layer, n_head=n_head, n_embd=n_embd
    )
    model = GPT(mconf)
    model.load_state_dict(torch.load(state_dict_file))
    print("model is loaded")
    tconf = TrainerConfig(num_workers=1)
    trainer = Trainer(model, train_dataset, None, tconf)
    
    for text_id in range(texts_count):
        context = [train_dataset.itos[np.random.randint(train_dataset.vocab_size)]]
        x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to(trainer.device)
        y = sample(model, x, text_len, temperature=1.0, sample=True, top_k=10)[0]
        completion = ' '.join([train_dataset.itos[int(i)] for i in y]).replace(' ##', '')
        yield completion

In [16]:
for x in generate_dataset(pathjoin(GENRE_DATA_DIR, LANG, 'A1.txt'), pathjoin(GPT_MODELS_DIR, LANG, 'A1')):
    print(x)

data has 463101 characters, 16955 unique.
model is loaded
In fact , a new global classroom and most of these students have by based on higher energy and materials . And I ' m pleased to say that far from global warming and mis is that I am I the United States of America and American mercenaries , we must also do something about how we can do to have cities and geopolitical considerations can not over the world . But I will ever give the way to an Iraqi / Iran about anything - Nonproliferation and geopolitical issues


1
